![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Construcción e implementación de modelos basados en Boosting
En este notebook aprenderá a construir e implementar modelos de Adaboost, Gradient Boosting y XGBoost. El primer modelo lo desarrollará de forma manual y usando la librería especializada sklearn, mientras que los otros dos los desarrollará solamente usando la librería.

## Instrucciones Generales:

Los modelos que construirá por medio de este notebook deberán predecir si un usuario deja o no de usar los servicios de una compañía (churn) teniendo en cuenta diferentes variables. Para conocer más detalles de la base de 'churn' puede ingresar al siguiente vínculo: http://srepho.github.io/Churn/Churn

Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 

In [ ]:
#!pip install pandas==2.2.3 numpy==2.0.2 matplotlib==3.9.4 scikit-learn==1.6.1 xgboost==2.1.4

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importar base de datos y librerías
import pandas as pd
import numpy as np

# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/churn.csv')

In [6]:
data

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False.
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False.
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False.
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False.


In [3]:
# Selección de variables numéricas (X)
X = data.iloc[:, [1,2,6,7,8,9,10]].astype(np.float64)
# Tranformación de variables booleanas a floats
X = X.join((data.iloc[:, [4,5]] == 'no').astype(np.float64))

# Definición variable de interés binaria (y)
y = (data.iloc[:, -1] == 'True.').astype(np.int64)

In [4]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)
n_samples = X_train.shape[0]

## Adaboost manual

In [7]:
# Definición de la cantidad de árboles de decisión del modelo
n_estimators = 10

# Definición de DataFrame para guardar pesos de las observaciones en cada árbol de decisión
weights = pd.DataFrame(index=X_train.index, columns=list(range(n_estimators)))

# Asignación los mismos pesos para todas las observaciones en el árbol 0
t = 0
weights[t] = 1 / n_samples

# Visualización de DataFrame de pesos
weights.head()

,0,1,2,3,4,5,6,7,8,9
2953,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2510,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Definición y entrenamiento (fit) del primer árbol de decisión (DecisionTreeClassifier)
trees = []
trees.append(DecisionTreeClassifier(max_depth=1))
trees[t].fit(X_train, y_train, sample_weight=weights[t].values)

DecisionTreeClassifier(max_depth=1)

In [9]:
# Estimación del error del primer árbol de decisión
y_pred_ = trees[t].predict(X_train)
error = []
error.append(1 - metrics.balanced_accuracy_score(y_pred_, y_train, sample_weight = weights[t].values))
error[t]

0.24114832535884934

In [10]:
# Cálculo del factor alpha del primer árbol de decisión
alpha = []
alpha.append(np.log((1 - error[t]) / error[t])/2)
alpha[t]

0.5731970670617188

In [11]:
# Actualización de los pesos a considerar en el segundo árbol (t+1)
weights[t + 1] = weights[t]
filter_ = y_pred_ != y_train

weights.loc[filter_, t + 1] = weights.loc[filter_, t] * np.exp(alpha[t])

In [12]:
# Normalización de los pesos
weights[t + 1] = weights[t + 1] / weights[t + 1].sum()

In [14]:
# Visualización de DataFrame de pesos
weights.head(20)

,0,1,2,3,4,5,6,7,8,9
2953,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2510,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2872,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2469,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1132,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Definición de loop que itera sobre todos los árboles definidos realizando los mismos cáculos presentados anteriormente
for t in range(1, n_estimators):
    # Definición y entrenamiento (fit) del árbol t
    trees.append(DecisionTreeClassifier(max_depth=1))
    trees[t].fit(X_train, y_train, sample_weight=weights[t].values)
    y_pred_ = trees[t].predict(X_train)
    # Estimación del error del árbol t
    error.append(1 - metrics.balanced_accuracy_score(y_pred_, y_train, sample_weight = weights[t].values))
    # Cálculo del factor alpha para el árbol t
    alpha.append(np.log((1 - error[t]) / error[t]) / 2)
    # Actualización de pesos para el árbol t+2
    weights[t + 1] = weights[t]
    filter_ = y_pred_ != y_train
    weights.loc[filter_, t + 1] = weights.loc[filter_, t] * np.exp(alpha[t])
    weights[t + 1] = weights[t + 1] / weights[t + 1].sum()

In [16]:
# Visualización de los errores de cada árbol
error

[0.24114832535884934,
 0.31085674971292176,
 0.26303609328491306,
 0.3509920019261563,
 0.3778663388376744,
 0.3908839674420268,
 0.436104330420925,
 0.4291168064851212,
 0.229599990460706,
 0.38855949425356906]

In [17]:
# Visualización de DataFrame de pesos
weights.head()

,0,1,2,3,4,5,6,7,8,9,10
2953,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000248,0.000267,0.000194,0.000221
617,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000248,0.000267,0.000194,0.000221
26,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000218,0.000204,0.000148,0.000169
853,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000248,0.000267,0.000194,0.000221
2510,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000248,0.000267,0.000194,0.000221


In [18]:
# Solo se usan modelos con error menor a 0.5
new_n_estimators = np.sum([x<0.5 for x in error])

In [19]:
# Prección sobre la muestra de test
y_pred_all = np.zeros((X_test.shape[0], new_n_estimators))
for t in range(new_n_estimators):
    y_pred_all[:, t] = trees[t].predict(X_test)

In [20]:
y_pred_all[:10,:]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
       [1., 0., 0., 1., 0., 1., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 1.]])

In [21]:
# Obtención de la predicción final al poderar las predicciones por el factor aplha
y_pred = (np.sum(y_pred_all * alpha[:new_n_estimators], axis=1) >= 1).astype(np.int64)

In [22]:
# Impresión del desempeño modelo
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.48184818481848185, 0.8572727272727273)

In [23]:
# Definición de un sólo árbol de decisón para compara el desempeño
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.32587859424920124, 0.8081818181818182)

## Adaboost usando sklearn

In [24]:
# Importación y definición de modelo AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf

AdaBoostClassifier()

In [25]:
# Entrenamiento (fit) y desempeño del modelo AdaBoostClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.36771300448430494, 0.8718181818181818)

## Gradient Boosting usando sklearn

In [26]:
# Importación y definición de modelo GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf

GradientBoostingClassifier()

In [27]:
# Entrenamiento (fit) y desempeño del modelo GradientBoostingClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.4644549763033175, 0.8972727272727272)

## XGBoost usando sklearn
Instalar la librería xgboost usando el comando: pip install xgboost. 

**Nota**: El entrenaimento (fit) del modelo XGBClassifier tarda varios minutos (aprox. 10 min) en ejecutarse en la plataforma de coursera, le recomendamos correrlo desde su computador en el repositorio clonado del curso.

In [30]:
# Importación y definición de modelo XGBClassifier
from xgboost import XGBClassifier
clf = XGBClassifier()
clf

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [31]:
# Entrenamiento (fit) y desempeño del modelo XGBClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.4521739130434782, 0.8854545454545455)